In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import deepchem as dc
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import wandb
import os
import torch.optim as optimizers

2021-08-23 17:23:50.297542: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt/lib:/opt/intel/tbb/lib:/opt/intel/clck/2019.0/lib:/opt/intel/compilers_and_libraries_2019/linux/lib:/opt/intel/compilers_and_libraries/linux/lib:/opt/intel/itac/2019.0.018/lib:/opt/intel/itac_2019/intel64/lib:/opt/intel/itac_latest/intel64/lib:/opt/intel/parallel_studio_xe_2019.0.045/clck_2019/lib:/opt/intel/parallel_studio_xe_2019.0.045/itac_2019/l

In [3]:
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import pickle
import torch
import torch.nn as nn
import tqdm
import copy
torch.multiprocessing.set_sharing_strategy('file_system') # this is important
# ulimit -n 500000
def set_worker_sharing_strategy(worker_id: int) -> None:
    torch.multiprocessing.set_sharing_strategy('file_system')

In [4]:
import sys
sys.path = ['/home/chrisw/Documents/projects/2021/graph-transformer/src'] + sys.path
from dfs_transformer import EarlyStopping, DFSCodeSeq2SeqFC, Deepchem2TorchGeometric, collate_minc_rndc_y

In [5]:
wandb.init(project='moleculenet', entity='chrisxx')

config = wandb.config
config.n_atoms = 118
config.n_bonds = 4
config.emb_dim = 120
config.nhead = 12
config.nlayers = 6
config.max_nodes = 400
config.max_edges = 600
config.dim_feedforward = 2048
config.lr = 0.0003
config.n_epochs = 25
config.patience = 3
config.factor = 0.95
config.minimal_lr = 6e-8
config.target_idx = 7
config.batch_size = 128
config.valid_patience = 5
config.valid_minimal_improvement = 0.00
config.pretrained_dir = '../models/chembl/transformer/medium/'
config.model_dir = '../../models/moleculenet/bbbp/medium/'
config.data_dir = "/mnt/ssd/datasets/ChEMBL/ChEMBL100_noH/"
config.num_workers = 4
config.load_last = False
config.dataset = 'bbbp' # 'bbbp', 'clintox', 'hiv', 'tox21'



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-08-23 17:23:55.386785: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt/lib:/opt/intel/tbb/

In [6]:
os.makedirs(config.model_dir, exist_ok=True)

In [7]:
torch.multiprocessing.get_all_sharing_strategies()

{'file_descriptor', 'file_system'}

In [8]:
path = config.data_dir

In [9]:
ngpu=1
device = torch.device('cuda:0' if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

In [10]:
to_cuda = lambda T: [t.cuda() for t in T]

In [11]:
model = DFSCodeSeq2SeqFC(n_atoms=config.n_atoms,
                         n_bonds=config.n_bonds, 
                         emb_dim=config.emb_dim, 
                         nhead=config.nhead, 
                         nlayers=config.nlayers, 
                         max_nodes=config.max_nodes, 
                         max_edges=config.max_edges,
                         atom_encoder=nn.Embedding(config.n_atoms, config.emb_dim), 
                         bond_encoder=nn.Linear(config.n_bonds, config.emb_dim))

In [12]:
if config.load_last:
    model.load_state_dict(torch.load(config.pretrained_dir+'checkpoint.pt'))

In [14]:
model.to(device)
if config.dataset == 'clintox':
    tasks, datasets, transformers = dc.molnet.load_clintox(reload=False, featurizer=dc.feat.RawFeaturizer(True))
elif config.dataset == 'tox21':
    tasks, datasets, transformers = dc.molnet.load_tox21(reload=False, featurizer=dc.feat.RawFeaturizer(True))
elif config.dataset == 'hiv':
    tasks, datasets, transformers = dc.molnet.load_hiv(reload=False, featurizer=dc.feat.RawFeaturizer(True))
elif config.dataset == 'bbbp':
    tasks, datasets, transformers = dc.molnet.load_bbbp(reload=False, featurizer=dc.feat.RawFeaturizer(True))

Failed to featurize datapoint 59, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
Failed to featurize datapoint 61, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
Failed to featurize datapoint 391, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
Failed to featurize datapoint 614, None. Appending empty array
Exception messa

In [15]:
model_head = nn.Linear(3*5*config.emb_dim, 1)
model_head.to(device)

Linear(in_features=1800, out_features=1, bias=True)

In [16]:
trainset, validset, testset = datasets
collate_fn = collate_minc_rndc_y
trainloader = DataLoader(Deepchem2TorchGeometric(trainset, -1), batch_size=config.batch_size, shuffle=True, pin_memory=False, collate_fn=collate_fn)
validloader = DataLoader(Deepchem2TorchGeometric(validset, -1), batch_size=config.batch_size, shuffle=False, pin_memory=False, collate_fn=collate_fn)
testloader = DataLoader(Deepchem2TorchGeometric(testset, -1), batch_size=config.batch_size, shuffle=False, pin_memory=False, collate_fn=collate_fn)

In [17]:
optim = optimizers.Adam(list(model.parameters()) + list(model_head.parameters()), lr=config.lr)

lr_scheduler = optimizers.lr_scheduler.ReduceLROnPlateau(optim, mode='min', verbose=True, patience=config.patience, factor=config.factor)
early_stopping_model = EarlyStopping(patience=config.valid_patience, delta=config.valid_minimal_improvement,
                              path=config.model_dir+'checkpoint_model.pt')
early_stopping_head = EarlyStopping(patience=config.valid_patience, delta=config.valid_minimal_improvement,
                              path=config.model_dir+'checkpoint_head.pt')
bce = torch.nn.BCEWithLogitsLoss()
sigmoid = nn.Sigmoid()

In [18]:
from sklearn.metrics import roc_auc_score, average_precision_score

def score(loader, model, model_head, use_min=False):
    val_roc = 0
    with torch.no_grad():
        full_preds, target = [], []
        for batch in tqdm.tqdm(loader):
            rndc, minc, z, eattr, y = batch
            if use_min:
                code = to_cuda(minc)
            else:
                code = to_cuda(rndc)
            y = y.to(device)
            features = model.encode(code, to_cuda(z), to_cuda(eattr)) # not clear whether to use minc or randc
            pred = sigmoid(model_head(features)).squeeze()
            
            pred_np = pred.detach().cpu().numpy().tolist()
            y_np = y.detach().cpu().numpy().tolist()
            full_preds += pred_np
            target += y_np

        target = np.asarray(target)
        full_preds = np.asarray(full_preds)
        roc = roc_auc_score(target, full_preds)
        prc = average_precision_score(target, full_preds)
    return roc, prc

In [19]:
valid_scores = []
try:
    for epoch in range(config.n_epochs):  
        epoch_loss = 0
        pbar = tqdm.tqdm(trainloader)
        model.train()
        for i, data in enumerate(pbar):
            optim.zero_grad()
            rndc, minc, z, eattr, y = data
            rndc = to_cuda(rndc)
            z = to_cuda(z)
            eattr = to_cuda(eattr)
            y = y.to(device)
            #prediction
            features = model.encode(rndc, z, eattr)
            prediction = model_head(features).squeeze()
            loss = bce(prediction, y)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optim.step()
            epoch_loss = (epoch_loss*i + loss.item())/(i+1)
            
            wandb.log({'loss':epoch_loss})
            pbar.set_description('Epoch %d: CE %2.6f'%(epoch+1, epoch_loss))
        model.eval()
        roc_auc_valid, prc_auc_valid = score(validloader, model, model_head) 
        valid_scores += [roc_auc_valid]
        lr_scheduler.step(epoch_loss)
        early_stopping_model(-roc_auc_valid, model)
        early_stopping_head(-roc_auc_valid, model_head)
        curr_lr = list(optim.param_groups)[0]['lr']
        wandb.log({'roc_valid':roc_auc_valid, 'prc_valid':prc_auc_valid, 'learning rate':curr_lr})
        print('ROCAUC',roc_auc_valid)

        if early_stopping_model.early_stop:
            break

        if curr_lr < config.minimal_lr:
            break

except KeyboardInterrupt:
    torch.save(model.state_dict(), config.model_dir+'_keyboardinterrupt.pt')
    print('keyboard interrupt caught')


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.45it/s]


ROCAUC 0.921389751552795


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.58it/s]


EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
ROCAUC 0.9106657608695652


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.10it/s]


...

EarlyStopping counter: 3 out of 5
EarlyStopping counter: 3 out of 5
ROCAUC 0.932113742236025


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.48it/s]


Epoch    10: reducing learning rate of group 0 to 2.8500e-04.
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 4 out of 5
ROCAUC 0.9266304347826086


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.32it/s]

EarlyStopping counter: 5 out of 5
EarlyStopping counter: 5 out of 5
ROCAUC 0.9273097826086957


In [20]:
model.load_state_dict(torch.load(config.model_dir+'checkpoint_model.pt'))
model_head.load_state_dict(torch.load(config.model_dir+'checkpoint_head.pt'))
model.eval()
roc_auc_valid, prc_auc_valid = score(testloader, model, model_head) 
wandb.log({'roc_test':roc_auc_valid, 'prc_test':prc_auc_valid})

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  6.20it/s]


In [21]:
print('ROC, PRC VALID', score_min(validloader, model, model_head, True))
print('ROC, PRC TEST', score_min(testloader, model, model_head, True))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.98it/s]


ROC_AUC_VALID 0.9244953416149068


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.46it/s]

ROC_AUC_TEST 0.6277579728297523
